In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import wandb

In [3]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
print(sys.path)

['C:\\Users\\Joshua\\PycharmProjects\\Machine_Learning_master', 'C:\\Users\\Joshua\\PycharmProjects\\Machine_Learning_master\\atlas-machine_learning\\personal_projects\\CryoET_obj_id', 'C:\\Users\\Joshua\\PycharmProjects\\Machine_Learning_master\\atlas-machine_learning\\supervised_learning\\object_detection', 'D:\\Apps\\PyCharm 2025.1\\plugins\\python-ce\\helpers\\pydev', 'D:\\Apps\\PyCharm 2025.1\\plugins\\python-ce\\helpers\\jupyter_debug', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\python310.zip', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\DLLs', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310', '', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib\\site-packages', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib\\site-packages\\win32', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Joshua\\anaconda3\\envs\\py310\\lib\\site

In [4]:
from models.baseline_model import build_generator, build_discriminator

ModuleNotFoundError: No module named 'models.baseline_model'

In [4]:
config_defaults = {
    "epochs": 20,
    "batch_size": 64,
    "latent_dim": 100,
    "learning_rate": 0.0002,
    "beta_1": 0.5
}
wandb.init(project="advanced_gan_celeba", config=config_defaults)
config = wandb.config

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joshuacox924007 (joshuacox924007-atlas-school) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
data_path = os.path.abspath(os.path.join('..', 'data', 'celeba_preprocessed.npz'))
data = np.load(data_path)
images = data['images']
print("Loaded images shape:", images.shape)

Loaded images shape: (202599, 64, 64, 3)


In [6]:
latent_dim = config.latent_dim
img_shape = (64, 64, 3)
epochs = config.epochs
batch_size = config.batch_size
num_batches = images.shape[0] // batch_size

In [7]:
generator = build_generator(latent_dim, k_size=5, filter_size=64, s1=2, s2=2, s3=2)
discriminator = build_discriminator(img_shape, k_size=5, alpha=0.2, s=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, beta_1=config.beta_1)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

discriminator.trainable = False
noise_input = tf.keras.Input(shape=(latent_dim,))
generated_image = generator(noise_input)
validity = discriminator(generated_image)
combined = tf.keras.Model(noise_input, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [8]:
def get_generated_images_figure(generator, epoch, examples=16, dim=(4,4), figsize=(8,8)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_imgs = generator.predict(noise)
    # Rescale images from [-1,1] to [0,1]
    generated_imgs = (generated_imgs + 1) / 2.0

    fig, axes = plt.subplots(dim[0], dim[1], figsize=figsize, sharex=True, sharey=True)
    cnt = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            axes[i, j].imshow(generated_imgs[cnt])
            axes[i, j].axis('off')
            cnt += 1
    fig.suptitle(f"Generated Images at Epoch {epoch}")
    plt.tight_layout()
    return fig

In [1]:
for epoch in range(1, epochs + 1):
    d_loss_epoch = 0.0
    g_loss_epoch = 0.0

    for batch in range(num_batches):
        # ---------------------
        #  Train Discriminator
        # ---------------------
        idx = np.random.randint(0, images.shape[0], batch_size)
        real_imgs = images[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = combined.train_on_batch(noise, valid)

        d_loss_epoch += d_loss[0]
        g_loss_epoch += g_loss

    d_loss_epoch /= num_batches
    g_loss_epoch /= num_batches

    print(f"Epoch {epoch}/{epochs} [D loss: {d_loss_epoch:.4f}] [G loss: {g_loss_epoch:.4f}]")
    wandb.log({"epoch": epoch, "d_loss": d_loss_epoch, "g_loss": g_loss_epoch})

    if epoch == 1 or epoch % 5 == 0:
        fig = get_generated_images_figure(generator, epoch)
        wandb.log({"generated_images": wandb.Image(fig, caption=f"Epoch {epoch}")})
        plt.close(fig)


NameError: name 'epochs' is not defined